In [1]:
import pyspark
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [15]:
from pyspark.sql import Row
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

plex_json = sqlContext.read.json('./data/Channels.json')

In [16]:
plex_json.printSchema()

root
 |-- MediaContainer: struct (nullable = true)
 |    |-- Directory: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- Location: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- id: long (nullable = true)
 |    |    |    |    |    |-- path: string (nullable = true)
 |    |    |    |-- agent: string (nullable = true)
 |    |    |    |-- allowSync: boolean (nullable = true)
 |    |    |    |-- art: string (nullable = true)
 |    |    |    |-- composite: string (nullable = true)
 |    |    |    |-- content: boolean (nullable = true)
 |    |    |    |-- contentChangedAt: long (nullable = true)
 |    |    |    |-- createdAt: long (nullable = true)
 |    |    |    |-- directory: boolean (nullable = true)
 |    |    |    |-- filters: boolean (nullable = true)
 |    |    |    |-- key: string (nullable = true)
 |    |    |    |-- language: string (nullable = true)
 |    |    |  

In [26]:
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql.functions import explode
df = plex_json.select(explode(plex_json.MediaContainer.Directory).alias("directory"))
# catalogs = df.select(explode(df.MediaContainer.Directory))

In [27]:
df.printSchema()

root
 |-- directory: struct (nullable = true)
 |    |-- Location: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- path: string (nullable = true)
 |    |-- agent: string (nullable = true)
 |    |-- allowSync: boolean (nullable = true)
 |    |-- art: string (nullable = true)
 |    |-- composite: string (nullable = true)
 |    |-- content: boolean (nullable = true)
 |    |-- contentChangedAt: long (nullable = true)
 |    |-- createdAt: long (nullable = true)
 |    |-- directory: boolean (nullable = true)
 |    |-- filters: boolean (nullable = true)
 |    |-- key: string (nullable = true)
 |    |-- language: string (nullable = true)
 |    |-- refreshing: boolean (nullable = true)
 |    |-- scannedAt: long (nullable = true)
 |    |-- scanner: string (nullable = true)
 |    |-- thumb: string (nullable = true)
 |    |-- title: string (nullable = true)
 |    |-- type: string (nullable = true)
 |    

In [30]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
get_paths = udf(lambda channel_loc: ','.join(list( map(lambda loc: loc.path, channel_loc) )), StringType())
catalog_mappings = df
catalog_mappings = catalog_mappings.withColumn("pathlist", get_paths(catalog_mappings.directory.Location))
catalog_mappings.printSchema()

root
 |-- directory: struct (nullable = true)
 |    |-- Location: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- path: string (nullable = true)
 |    |-- agent: string (nullable = true)
 |    |-- allowSync: boolean (nullable = true)
 |    |-- art: string (nullable = true)
 |    |-- composite: string (nullable = true)
 |    |-- content: boolean (nullable = true)
 |    |-- contentChangedAt: long (nullable = true)
 |    |-- createdAt: long (nullable = true)
 |    |-- directory: boolean (nullable = true)
 |    |-- filters: boolean (nullable = true)
 |    |-- key: string (nullable = true)
 |    |-- language: string (nullable = true)
 |    |-- refreshing: boolean (nullable = true)
 |    |-- scannedAt: long (nullable = true)
 |    |-- scanner: string (nullable = true)
 |    |-- thumb: string (nullable = true)
 |    |-- title: string (nullable = true)
 |    |-- type: string (nullable = true)
 |    

In [31]:
catalog_mappings.show(5)

+--------------------+--------------------+
|           directory|            pathlist|
+--------------------+--------------------+
|[[[18, /mnt/nas/v...|/mnt/nas/video/mo...|
|[[[17, /mnt/nas/v...|/mnt/nas/video/TV...|
|[[[15, /mnt/nas/v...|/mnt/nas/video/in...|
|[[[22, /mnt/nas/v...|/mnt/nas/video/Au...|
|[[[2, /mnt/nas/vi...|/mnt/nas/video/ki...|
+--------------------+--------------------+
only showing top 5 rows

